In [5]:
import os 
import random
import queue
import urllib.parse
import time
import urllib.request
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urldefrag
from reppy.cache import RobotsCache
import pyodbc

In [9]:
BACKQUEUES= 3
THREADS= BACKQUEUES*3
FRONTQUEUES= 5
WAITTIME= 15 
out_file_path="C:/Users/ahmed/Desktop/out_URLs.txt"
cache = RobotsCache(500)
db_link=False
crawled = []

NameError: name 'conn' is not defined

In [ ]:
def out_file_handler():
    if not os.path.isfile(out_file_path):
            out_file = open(out_file_path,'w')
            return out_file
        

In [56]:
class frontier:
    # add the code for frontier here
    # should have functions __init__, get_URL, add_URLs, add_to_backqueue

    ######### add_to_backQueue ###############################

    def add_to_backqueue(self, pos):
        counter = 0
        checker = False
        flag = False
        while checker == False and counter < 3:
            flag = False
            FQ = random.randint(1, FRONTQUEUES)
            FQ = FQ - 1
            if not self.FrontQueues[FQ].empty():
                URL = self.FrontQueues[FQ].get()
                for i in range(len(self.BackQueues)):
                    if i != pos and not self.BackQueues[i].empty():
                        d = self.BackQueues[i].queue
                        url_existing = urllib.parse.urlparse(d[0])
                        url_adding = urllib.parse.urlparse(URL)
                        if url_existing.netloc == url_adding.netloc:
                            self.BackQueues[i].put(URL)
                            flag = True
                if flag == False:
                    self.BackQueues[pos].put(URL)
                    checker = True
            counter = counter + 1

        if checker == False:
            for j in range(len(self.FrontQueues)):
                if not self.FrontQueues[j].empty():
                    URL = self.FrontQueues[j].get()
                    for i in range(len(self.BackQueues)):
                        if i != pos and not self.BackQueues[i].empty():
                            d = self.BackQueues[i].queue
                            url_existing = urllib.parse.urlparse(d[0])
                            url_adding = urllib.parse.urlparse(URL)
                            if url_existing.netloc == url_adding.netloc:
                                self.BackQueues[i].put(URL)
                                flag = True
                    if flag == False:
                        self.BackQueues[pos].put(URL)

                        break

    ############# _INIT_ #############
    def __init__(self):
        '''
        Initializer
        '''
        self.heap = []
        self.FrontQueues = []
        for i in range(FRONTQUEUES):
            self.FrontQueues.append(queue.Queue())

        self.BackQueues = []
        for i in range(BACKQUEUES):
            self.BackQueues.append(queue.Queue())

        fpath = "C:/Users/ahmed/Desktop/Seed URLs.txt"
        if not os.path.isfile(fpath):
            print("Seed URLs File does not exist")
        else:
            with open(fpath, "r") as file:
                for line in file:
                    url = line.strip()
                    i = prioritizer(url, FRONTQUEUES)
                    self.FrontQueues[i - 1].put(url)

    ######## get_URL########################

    def get_URL(self):
        index_min = min(range(len(self.heap)), key=self.heap.__getitem__)
        url = self.BackQueues[index_min].get()
        crawled.append(url)
        if self.BackQueues[index_min].empty():
            self.add_to_backqueue(index_min)
            d = self.BackQueues[index_min].queue
            print(len(d))
            url_existing = urllib.parse.urlparse(d[0])
            url_adding = urllib.parse.urlparse(url)
            if url_existing.netloc == url_adding.netloc:
                self.heap[index_min] == self.heap[index_min] + 15
        else:
            self.heap[index_min] == self.heap[index_min] + 15
        return url

    ########### add_URLs ###################

    def add_URLs(self, URLs):
        if len(URLs) > 0:
            for i in URLs:
                self.FrontQueues[prioritizer(i, FRONTQUEUES)].put(i)

    ######## prep_BackQueue#################
    def prep_BackQueue(self):

        for i in range(len(self.BackQueues)):
            self.add_to_backqueue(i)
            self.heap.insert(i, 0)


def prioritizer(URL, f):
    """
    Take URL and returns priority from 1 to F
    Right now it like a stub function.
    It will return a random number from 1 to f for given inputs.
    """

    return random.randint(1, f)

In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-PRRQ2CQ;'
                      'Database=Crawler;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
cursor.execute('CREATE TABLE Crawled (URL varchar(100), Data TEXT)')

In [17]:
def fetch(url):
    w=urllib.request.urlopen(url)
    data=w.read()
    data_str = data.decode('utf-8')
    cursor.execute('''
                INSERT INTO Crawled (URL, Data)
                VALUES
                ('%s',%s)
                
                '''%(url,data_str))
    
    return data

In [15]:
def parse(data,url):
    soup=BeautifulSoup(data,'html.parser')
    urls=[]
    for i in soup.find_all("a"):
        link = i.get("href")
        if isUrlAbsolute(link):
            link=urljoin(url,link)
        urls.append(link)
    return urls

In [11]:
def isUrlAbsolute(url):
    return ("https:" not in url and "http:" not in url)

In [ ]:
def urlFilter(urls):
    filtered = []
    for i in urls:
        if cache.allowed(i, 'foo'):
            filtered.append(i)
    return filtered
        

In [6]:
from urllib.parse import urljoin, urldefrag
base_url = 'http://en.wikipedia.org/wiki/Main_Page'
relative_url = '/wiki/Wikipedia:General_disclaimer'
url = urljoin(base_url, relative_url)
print (url)
ux = urldefrag(url)
print(ux)
print (ux[0])

http://en.wikipedia.org/wiki/Wikipedia:General_disclaimer
DefragResult(url='http://en.wikipedia.org/wiki/Wikipedia:General_disclaimer', fragment='')
http://en.wikipedia.org/wiki/Wikipedia:General_disclaimer


In [7]:
db_connect()

In [11]:
import pyodbc 
connection = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-PRRQ2CQ;'
                      'Database=test_db;'
                      'Trusted_Connection=yes;')


In [12]:
cur = connection.cursor()


In [15]:
connection.commit()

In [14]:

a = "www.google.com"
b = "Minnn"

cur.execute('''
                INSERT INTO table_name (URL, Data)
                VALUES
                ('%s','%s')
                
                '''%(a,b))

In [54]:
conn

<class 'bytes'>
<class 'str'>


In [14]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://en.wikipedia.org/robots.txt")
rp.read()
rrate = rp.request_rate("*")

rp.can_fetch("*", "/wiki/Wikipedia:Vandalensperrung/")




True

In [9]:
from advertools import robotstxt_test
robotstxt_test('https://en.wikipedia.org/wiki/Wikipedia:Vandalensperrung/', user_agents=['*'], urls=[''])

,robotstxt_url,user_agent,url_path,can_fetch
0,https://en.wikipedia.org/robots.txt,*,/wiki/Wikipedia:Vandalensperrung/,False


In [23]:

cache = RobotsCache(100)
cache.allowed('https://en.wikipedia.org/wiki/Wikipedia:Vandalensperrung/', 'foo')



False

In [26]:
cache.allowed('https://en.wikipedia.org', 'foo')

True

10
10
